# Regridding demo datesets using regrid_to_healpix
Before starting this notebook, Create temporaly small data set using the noteoobk 'prep_regrid.ipynb'
To install the regrid_to_healpix tool, do

```pip install "git+https://github.com/EOPF-DGGS/regrid_to_healpix.git"```

Later, this should be tested by loading the full virtualizarr file indicated in the prep_regrid.ipynb

In [1]:
import xarray as xr
import numpy as np
import xdggs

In [2]:
xr.open_zarr('/Users/todaka/data/RIOMAR/small.zarr')

<xarray.Dataset> Size: 2MB
Dimensions:       (time_counter: 2, s_rho: 40, y_rho: 50, x_rho: 68)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 16B 2023-12-01T00:48:10 2023-...
    time_instant  (time_counter) datetime64[ns] 16B dask.array<chunksize=(2,), meta=np.ndarray>
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * y_rho         (y_rho) float32 200B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho         (x_rho) float32 272B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    mask          (y_rho, x_rho) bool 3kB dask.array<chunksize=(50, 68), meta=np.ndarray>
    nav_lat_rho   (y_rho, x_rho) float32 14kB dask.array<chunksize=(50, 68), meta=np.ndarray>
    nav_lon_rho   (y_rho, x_rho) float32 14kB dask.array<chunksize=(50, 68), meta=np.ndarray>
Data variables:
    salt          (time_counter, s_rho, y_rho, x_rho) float32 1MB dask.array<chunksize=(1, 40, 50, 68), meta=np.ndarray>
    temp          (time_counter, s_rho, y_rho, x_rho) float32 1MB dask.array<chunksize=(1, 40, 50, 68), meta=np.ndarray>
    zeta          (time_counter, y_rho, x_rho) float32 27kB dask.array<chunksize=(1, 50, 68), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [3]:
small_ds=xr.open_zarr('/Users/todaka/data/RIOMAR/small.zarr')
#small_ds=xr.open_zarr('/Users/todaka/data/RIOMAR/very_small.zarr') #only temp
#ds = small_ds[["temp", "nav_lon_rho", "nav_lat_rho"]].isel(s_rho=0).stack(point=("y_rho","x_rho")).dropna(dim="point") #only drop the z coordinate 
#ds = small_ds[["temp","nav_lon_rho", "nav_lat_rho"]].stack(point=("y_rho","x_rho")).dropna(dim="point") #only temp
ds = small_ds.stack(point=("y_rho","x_rho")).dropna(dim="point")
ds

<xarray.Dataset> Size: 1MB
Dimensions:       (time_counter: 2, s_rho: 40, point: 2193)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 16B 2023-12-01T00:48:10 2023-...
    time_instant  (time_counter) datetime64[ns] 16B dask.array<chunksize=(2,), meta=np.ndarray>
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * point         (point) object 18kB MultiIndex
    mask          (point) bool 2kB dask.array<chunksize=(2193,), meta=np.ndarray>
    nav_lat_rho   (point) float32 9kB dask.array<chunksize=(2193,), meta=np.ndarray>
    nav_lon_rho   (point) float32 9kB dask.array<chunksize=(2193,), meta=np.ndarray>
  * y_rho         (point) float32 9kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho         (point) float32 9kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Data variables:
    salt          (time_counter, s_rho, point) float32 702kB dask.array<chunksize=(1, 40, 2193), meta=np.ndarray>
    temp          (time_counter, s_rho, point) float32 702kB dask.array<chunksize=(1, 40, 2193), meta=np.ndarray>
    zeta          (time_counter, point) float32 18kB dask.array<chunksize=(1, 2193), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [4]:
from regrid_to_healpix.regrid_to_healpix_bilinear import Set
child_level=13
# Build operator once
lon = ds["nav_lon_rho"].values.astype(np.float64)
lat = ds["nav_lat_rho"].values.astype(np.float64)

nr = Set(lon_deg=lon, lat_deg=lat, level=child_level, device="cpu", threshold=0.5, ellipsoid="WGS84")
cell_ids = np.asarray(nr.get_cell_ids(), dtype=np.int64)
ncell = int(cell_ids.size)

def to_healpix_point(data_1d):
    out = nr.transform(np.asarray(data_1d, dtype=np.float64), lam=0.1)
    return np.asarray(out, dtype=np.float64)

# Apply to the whole Dataset: only to chosen data_vars
vars_to_regrid = ["temp"]  # add "salt", "zeta", ...
ds_in = ds#[vars_to_regrid]

ds_hp = xr.apply_ufunc(
    to_healpix_point,
    ds_in,
    input_core_dims=[["point"]],
    output_core_dims=[["cell_ids"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float64],
    dask_gufunc_kwargs={"output_sizes": {"cell_ids": ncell}},
    keep_attrs=True,  # keep dataset + variable attrs where possible
)

# Re-attach coordinate + its metadata
ds_hp = ds_hp.assign_coords(cell_ids=("cell_ids", cell_ids))
ds_hp["cell_ids"].attrs.update({
    "grid_name": "healpix",
    "level": 13,
    "indexing_scheme": "nested",
    "ellipsoid": "WGS84",
})

ds_hp

/Users/todaka/python/git/regrid_to_healpix/regrid_to_healpix/regrid_to_healpix_bilinear.py:74: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:51.)
  self.M  = M_coo.to_sparse_csr()


<xarray.Dataset> Size: 5MB
Dimensions:       (time_counter: 2, s_rho: 40, cell_ids: 3504)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 16B 2023-12-01T00:48:10 2023-...
    time_instant  (time_counter) datetime64[ns] 16B dask.array<chunksize=(2,), meta=np.ndarray>
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * cell_ids      (cell_ids) int64 28kB 224374237 224374239 ... 225780884
Data variables:
    salt          (time_counter, s_rho, cell_ids) float64 2MB dask.array<chunksize=(1, 40, 3504), meta=np.ndarray>
    temp          (time_counter, s_rho, cell_ids) float64 2MB dask.array<chunksize=(1, 40, 3504), meta=np.ndarray>
    zeta          (time_counter, cell_ids) float64 56kB dask.array<chunksize=(1, 3504), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [5]:
ds_hp.compute().pipe(xdggs.decode).dggs.assign_latlon_coords().dggs.explore(alpha=0.3)

reindex(cell_ids=target_ids) builds a new dataset whose cell_ids coordinate is exactly target_ids (your aligned_child_ids), in that order. So:
	•	If a cell_id exists in ds_hp but is not in target_ids → it will not appear in ds_aligned (it’s removed).
	•	If a cell_id is in target_ids and also in ds_hp → its data is kept (and aligned to the right position).
	•	If a cell_id is in target_ids but missing from ds_hp → that position is created and filled with NaN.

So reindex is effectively “take this target index; keep what matches; add missing as NaN; drop everything else”.

In [6]:
import healpix_geo
# load
parent_level=10
parent_ids = np.load("parent_ids.npy")  # dtype preserved (uint64)

aligned_child_ids = np.unique(healpix_geo.nested.zoom_to(
    parent_ids,
    depth=parent_level,
    new_depth=child_level
))
                              
# Make sure types match (important: your ds_hp cell_ids look like int64)
target_ids = aligned_child_ids.astype(ds_hp["cell_ids"].dtype)
chunk_size=4**(child_level - parent_level )

ds_aligned = ds_hp.reindex(cell_ids=target_ids, fill_value=np.nan).chunk({"cell_ids": chunk_size})
ds_aligned

<xarray.Dataset> Size: 6MB
Dimensions:       (s_rho: 40, time_counter: 2, cell_ids: 4544)
Coordinates:
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * time_counter  (time_counter) datetime64[ns] 16B 2023-12-01T00:48:10 2023-...
    time_instant  (time_counter) datetime64[ns] 16B dask.array<chunksize=(2,), meta=np.ndarray>
  * cell_ids      (cell_ids) int64 36kB 224374592 224374593 ... 225780799
Data variables:
    salt          (time_counter, s_rho, cell_ids) float64 3MB dask.array<chunksize=(1, 40, 64), meta=np.ndarray>
    temp          (time_counter, s_rho, cell_ids) float64 3MB dask.array<chunksize=(1, 40, 64), meta=np.ndarray>
    zeta          (time_counter, cell_ids) float64 73kB dask.array<chunksize=(1, 64), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [7]:
%%time
ds_aligned.to_zarr('hp.zarr',consolidated=True,mode='w')

/Users/todaka/micromamba/envs/pangeo/lib/python3.13/site-packages/zarr/api/asynchronous.py:247: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


CPU times: user 507 ms, sys: 339 ms, total: 846 ms
Wall time: 575 ms


In [8]:
(
    ds_aligned.compute()
    .pipe(xdggs.decode)
    .dggs.assign_latlon_coords()
    .dggs.explore(alpha=0.3)
)

In [9]:
new_ds=ds_aligned
display((new_ds.max() - ds.max()).compute(),
(new_ds.min() - ds.min()).compute(),
(new_ds.mean() - ds.mean()).compute())

<xarray.Dataset> Size: 24B
Dimensions:  ()
Data variables:
    salt     float64 8B -0.002832
    temp     float64 8B -0.005021
    zeta     float64 8B -4.768e-05
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

<xarray.Dataset> Size: 24B
Dimensions:  ()
Data variables:
    salt     float64 8B 0.6243
    temp     float64 8B -9.243
    zeta     float64 8B 0.004278
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

<xarray.Dataset> Size: 24B
Dimensions:  ()
Data variables:
    salt     float64 8B 0.04624
    temp     float64 8B -0.003944
    zeta     float64 8B -0.001716
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...